# Análise de sentimento usando word embeddings - IMDB

Anteriormente vimos uma primeira solução de análise de sentimento utilizando *bag of words*.
Agora iremos ilustrar o uso de *word embeddings* como vetor de atributos latentes de cada palavra.

Duas soluções são propostas neste exercícios:

1. Utilizando rede neural com camadas densas
2. Utilizando camadas convolucionais 1D

Diferentemente da solução apresentada com *bag of words*, nestas duas soluções, é necessário que o
número de palavras seja o mesmo para cada amostra. Para isso, limita-se o número de palavras e caso
o número de palavras for menor, completa-se com um código especial e palavras além do limite são
descartadas.

## Importação dos pacotes

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os, sys
import json
import numpy as np
import pandas as pd
import numpy.random as nr

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR, StepLR
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable

from torchvision import datasets, transforms, models

import lib.pytorch_trainer as ptt

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)

GPU available: False


## Dataset IMDB

### Lendo do disco

O dataset é composto de 25 mil amostras de treinamento e 25 mil amostras de teste.
Cada amostra possui um texto de tamanho que varia entre 11 e 2494 palavras. 
Cada amostra tem um rótulo igual a 1 para denominar sentimento positivo e 0 para sentimento negativo.

In [2]:
word_index = json.load(open('/data/datasets/IMDB/imdb_word_index.json'))
data = np.load('/data/datasets/IMDB/imdb.npz')
x_test, x_train, y_train, y_test = data['x_test'], data['x_train'], data['y_train'], data['y_test']

n_words = len(word_index)
n_train = x_train.shape[0]
n_test  = x_test.shape[0]

word_list = [None for i in range(n_words+1)]
for k, v in word_index.items():
    word_list[v] = k

n_words, n_train, n_test

(88584, 25000, 25000)

In [3]:
def print_stats(x_train, x_test, word_list=None):
    print('Train word index limits:', min([min(s) for s in x_train]), max([max(s) for s in x_train]))
    print('Test word index limits:', min([min(s) for s in x_test]), max([max(s) for s in x_test]))
    print('\nTrain sequence length limits:', min([len(x) for x in x_train]), max([len(x) for x in x_train]))
    print('Test sequence length limits:', min([len(x) for x in x_test]), max([len(x) for x in x_test]))
    if word_list:
        print('\nMost frequent words:', word_list[1:11])
    
print_stats(x_train, x_test, word_list)

Train word index limits: 1 88584
Test word index limits: 1 88581

Train sequence length limits: 10 2493
Test sequence length limits: 6 2314

Most frequent words: ['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']


### Limitando o vocabulário

Retiramos das sequências as palavras com índice maior que o valor especificado em `voc_size`.

In [4]:
voc_size = 5000

xtra = [[w for w in x if (w < voc_size)] for x in x_train]
xval = [[w for w in x if (w < voc_size)] for x in x_test]
print_stats(xtra, xval)

Train word index limits: 1 4999
Test word index limits: 1 4999

Train sequence length limits: 9 1973
Test sequence length limits: 6 2113


### Obtendo sequências de mesmo comprimento

Fazemos com que todas as sequências tenham o mesmo comprimento, especificado em `seq_len`. As sequências mais longas que `seq_len` são truncadas e as menores, completadas com zeros.

In [5]:
def pad_sequences(sequences, seq_len, post_pad=True, fill_value=0):
    new_seq = []
    for seq in sequences:
        n = len(seq)
        if n > seq_len:
            if post_pad:
                new_seq.append(seq[-seq_len:])
            else:
                new_seq.append(seq[:seq_len])
        else:
            zseq = [fill_value for i in range(seq_len)]
            if post_pad:
                zseq[-n:] = seq
            else:
                zseq[:n] = seq
            new_seq.append(zseq)
    return new_seq
    

In [6]:
seq_len = 500
xtra = pad_sequences(xtra, seq_len, post_pad=True)
xval = pad_sequences(xval, seq_len, post_pad=True)
print_stats(xtra, xval)

Train word index limits: 0 4999
Test word index limits: 0 4999

Train sequence length limits: 500 500
Test sequence length limits: 500 500


### Convertendo para tensores

In [7]:
Xtrain = torch.from_numpy(np.array(xtra, np.int))
Xvalid = torch.from_numpy(np.array(xval, np.int))
ytrain = torch.from_numpy(np.array(y_train, np.int))
yvalid = torch.from_numpy(np.array(y_test, np.int))

Xtrain.size(), Xtrain.max(), Xvalid.size(), Xvalid.max()

(torch.Size([25000, 500]), 4999, torch.Size([25000, 500]), 4999)

## Rede Densa


In [8]:
class MySimpleNet(nn.Module):

    def __init__(self, seq_len=seq_len, voc_size=voc_size, embed_dim=None):
        super().__init__()
        self.flat_size = seq_len * embedding_dim        
        self.emb = nn.Embedding(voc_size, embed_dim)
        nn.init.xavier_uniform(self.emb.weight)
        self.fc1 = nn.Linear(self.flat_size, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = self.emb(x)
        x = x.view(-1, self.flat_size)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)
        x = self.fc2(x)
        return x


### Treinamento

In [9]:
trainIt = False
resetIt = False

embedding_dim = 50
batch_size = 100
n_epochs = 10

# Callbacks
# ---------
state_fn = '../../models/sentimento_1'
accuracy_cb = ptt.AccuracyMetric()
chkpt_cb = ptt.ModelCheckpoint(state_fn, reset=resetIt, verbose=1)
print_cb = ptt.PrintCallback()
plot_cb = ptt.PlotCallback()

# Model, optimizer and learning rate scheduler
# --------------------------------------------
model = MySimpleNet(seq_len, voc_size, embedding_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

# Network trainer
# ---------------
training_parameters = {
    'model':         model, 
    'criterion':     nn.CrossEntropyLoss(),
    'optimizer':     optimizer, 
    'lr_scheduler':  scheduler, 
    'callbacks':     [accuracy_cb, chkpt_cb, print_cb],
}
trainer = ptt.DeepNetTrainer(**training_parameters)


In [10]:
if trainIt:
    trainer.fit(n_epochs, Xtrain, ytrain, valid_data=(Xvalid, yvalid), batch_size=batch_size)
else:
    print('\nTraining disabled.\nThis model was trained for {} epochs.'.format(trainer.last_epoch))


Training disabled.
This model was trained for 0 epochs.


#### Treinamento em CPU (AWS c4x.2large, _compute optimized 8 cores_ ):
`
Start training for 10 epochs
  1:  12.8s   T: 0.67543 0.57752   V: 0.59997 0.73932 best
  2:  13.1s   T: 0.41619 0.83124   V: 0.33354 0.86456 best
  3:  16.8s   T: 0.27492 0.89132   V: 0.29643 0.87532 best
  4:  13.1s   T: 0.23069 0.91264   V: 0.28460 0.87956 best
  5:  13.1s   T: 0.19852 0.92964   V: 0.27674 0.88468 best
  6:  22.7s   T: 0.17766 0.93948   V: 0.27637 0.88312 best
  7:  96.0s   T: 0.16022 0.94736   V: 0.27513 0.88440 best
  8: 116.6s   T: 0.14364 0.95680   V: 0.27759 0.88304 
  9: 106.3s   T: 0.12834 0.96488   V: 0.28848 0.87756 
 10: 100.8s   T: 0.11256 0.97276   V: 0.28174 0.88176 
Best model was saved at epoch 7 with loss 0.27513: ../../models/sentimento_1
Stop training at epoch: 10/10
`
#### Treinamento em GPU (GTX1080 Ti, 8GB):
`
  1:   5.8s   T: 0.68932 0.53556   V: 0.65994 0.63064 best
...
`

### Avaliação

In [11]:
if 'ModelCheckpoint' in [cb.__class__.__name__ for cb in trainer.callbacks]:
    trainer.load_state(state_fn)

rmetrics = trainer.evaluate(Xtrain, ytrain, metrics=[accuracy_cb])
print('Model training set accuracy after training: {:.5f}'.format(rmetrics['acc']))
print()
rmetrics = trainer.evaluate(Xvalid, yvalid, metrics=[accuracy_cb])
print('Model validation set accuracy after training: {:.5f}'.format(rmetrics['acc']))

evaluate: 2499/2499 ok
Model training set accuracy after training: 0.95920

evaluate: 2499/2499 ok
Model validation set accuracy after training: 0.88344


## Rede convolucional

In [12]:
class MyNet(nn.Module):
    
    def __init__(self, seq_len=seq_len, voc_size=voc_size, embed_dim=embedding_dim, 
                 n_conv_filters=128, conv_kernel_size=5):
        super().__init__()
        
        k = conv_kernel_size - 1
        n = (((seq_len - k) // 2 - k) // 2 - k) // 2
        self.flat_size = n * n_conv_filters
        
        self.embedding = nn.Embedding(voc_size, embed_dim)
        nn.init.xavier_uniform(self.embedding.weight)

        self.conv_net = nn.Sequential(
            nn.Conv1d(embed_dim, n_conv_filters, conv_kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.5),
            
            nn.Conv1d(n_conv_filters, n_conv_filters, conv_kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.5),
            
            nn.Conv1d(n_conv_filters, n_conv_filters, conv_kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.5),
        )
        
        self.fc_net = nn.Sequential(
            nn.Linear(self.flat_size, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2),
        )

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2)
        x = self.conv_net(x)
        x = x.view(-1, self.flat_size)
        x = self.fc_net(x)
        return x

### Treinamento

In [13]:
trainIt = False
resetIt = False

# embedding_dim = 50
# batch_size = 100
# n_epochs = 15

# Callbacks
# ---------
state_fn = '../../models/sentimento_3'
accuracy_cb = ptt.AccuracyMetric()
chkpt_cb = ptt.ModelCheckpoint(state_fn, reset=resetIt, verbose=1)
print_cb = ptt.PrintCallback()
plot_cb = ptt.PlotCallback()

# Model, optimizer and learning rate scheduler
# --------------------------------------------
model = MyNet(seq_len, voc_size, embedding_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

# Network trainer
# ---------------
training_parameters = {
    'model':         model, 
    'criterion':     nn.CrossEntropyLoss(),
    'optimizer':     optimizer, 
    'lr_scheduler':  scheduler, 
    'callbacks':     [accuracy_cb, chkpt_cb, print_cb],
}
trainer = ptt.DeepNetTrainer(**training_parameters)


In [14]:
if trainIt:
    trainer.fit(n_epochs, Xtrain, ytrain, valid_data=(Xvalid, yvalid), batch_size=batch_size)
else:
    print('\nTraining disabled.\nThis model was trained for {} epochs.'.format(trainer.last_epoch))


Training disabled.
This model was trained for 0 epochs.


#### Treinamento em GPU (GTX1080 Ti, 8GB):
`
Start training for 10 epochs
  1:  10.0s   T: 0.51950 0.69276   V: 0.30188 0.87500 best
  2:   9.7s   T: 0.26000 0.89792   V: 0.26565 0.88956 best
  3:   9.7s   T: 0.20300 0.92276   V: 0.27181 0.88824 
  4:   9.8s   T: 0.16678 0.93864   V: 0.29407 0.88728 
  5:   9.8s   T: 0.12640 0.95380   V: 0.31985 0.87868 
  6:   9.7s   T: 0.09621 0.96600   V: 0.35878 0.87376 
  7:   9.7s   T: 0.08262 0.97012   V: 0.41535 0.87944 
  8:   9.7s   T: 0.05677 0.98076   V: 0.48370 0.87468 
  9:   9.7s   T: 0.04682 0.98400   V: 0.53459 0.87464 
 10:   9.8s   T: 0.02949 0.98916   V: 0.62111 0.87328 
Best model was saved at epoch 2 with loss 0.26565: ../../models/sentimento_3
Stop training at epoch: 10/10
`

### Avaliação

In [15]:
if 'ModelCheckpoint' in [cb.__class__.__name__ for cb in trainer.callbacks]:
    trainer.load_state(state_fn)

rmetrics = trainer.evaluate(Xtrain, ytrain, metrics=[ptt.AccuracyMetric()])
print('Model training set accuracy after training: {:.5f}'.format(rmetrics['acc']))
print()
rmetrics = trainer.evaluate(Xvalid, yvalid, metrics=[ptt.AccuracyMetric()])
print('Model validation set accuracy after training: {:.5f}'.format(rmetrics['acc']))

evaluate: 2499/2499 ok
Model training set accuracy after training: 0.93916

evaluate: 2499/2499 ok
Model validation set accuracy after training: 0.88956


## Resumo dos resultados

1. Experimento *bag of words*: 87% de acurácia
2. Experimento *word embeddings*, rede densa: 88%
3. Experimento *word embeddings*, rede convolucional: 89%